In [180]:
import torch
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F

In [181]:
names = open("names.txt", "r").read().splitlines()
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(stoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [182]:
def build_dataset(names):
    block_size = 3 #context length of how many chars we take to predict the next char
    X, Y = [],[]
    
    for w in names:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # rolling window, makes sense when you run it
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

In [183]:
import random
random.seed(42)
random.shuffle(names)
X_train, Y_train = build_dataset(names[:int(0.8*len(names))]) #get first 80% of the dataset
X_valid, Y_valid = build_dataset(names[int(0.8*len(names)):int(0.9*len(names))]) # get between 80%-90%
X_test, Y_test = build_dataset(names[int(0.9*len(names)):]) # get last 10% of dataset

block_size = 3
vocab_size = len(itos)
#[itos[x[2].item()] for x in X_train[:5]]

In [184]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_emb), generator=g)

#Layer 1
W1 = torch.randn((n_emb * block_size, n_hidden) , generator = g) * (5/3)/(n_emb * block_size)**0.5
b1 = torch.randn(n_hidden, generator = g) * 0.1

#Layer2
W2 = torch.randn((n_hidden, vocab_size) , generator = g) * 0.1
b2 = torch.randn(vocab_size, generator = g) * 0.1

#BatchNorm params
gamma = torch.randn((1, n_hidden)) * 0.1 + 1.0
beta = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b1, gamma, beta]
print(f"number of all params: {sum(p.numel() for p in parameters)}")

number of all params: 4174


In [185]:
batch_size = 32
n = batch_size
# get a mini-batch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator = g)
Xbatch, Ybatch = X_train[ix], Y_train[ix]

In [186]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [187]:
# forward pass, but in smaller steps because i'll need to take gradients of each step
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [188]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
Xb, Yb = X_train[ix], Y_train[ix] # batch X,Y

In [189]:
emb = C[Xb] # embed the characters into vectors
print(C.shape, Xb)
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
#print(logprobs.shape)
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

torch.Size([27, 10]) tensor([[ 1,  1,  4],
        [18, 14,  1],
        [11,  5,  9],
        [ 0,  0,  1],
        [12, 15, 14],
        [ 0, 17,  1],
        [ 0,  0, 13],
        [13, 13,  1],
        [ 8, 25, 12],
        [ 0,  0, 26],
        [22, 15, 14],
        [19, 13,  9],
        [ 0,  0,  0],
        [ 0,  4,  5],
        [ 5, 14,  9],
        [18,  5,  5],
        [ 0,  4,  1],
        [ 1, 18,  1],
        [ 0,  0,  0],
        [ 0,  5, 12],
        [ 0, 10,  1],
        [ 9, 14,  1],
        [ 0,  0,  0],
        [ 0,  0, 18],
        [20,  5,  1],
        [ 0, 11, 15],
        [ 0,  0,  7],
        [ 0, 18,  5],
        [26,  5, 18],
        [ 0,  0, 14],
        [ 3,  5, 14],
        [ 0, 18, 15]])


tensor(3.3323, grad_fn=<NegBackward0>)

In [190]:
# manual backprop through everything. Split BatchNorm and Cross-Entropy into a million small pieces which we will backpropagate through
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
dprobs = dlogprobs * (1.0/probs)
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogits = (torch.ones_like(logits) * dnorm_logits)
dlogit_maxes = -(dnorm_logits).sum(1, keepdims=True)
#print(dlogit_maxes.shape)
dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1]) * dlogit_maxes
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1-h**2) * dh
dbngain = (bnraw * dhpreact).sum(0)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0)
dbndiff = dbnraw * bnvar_inv
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)
dbnvar = dbnvar_inv * -0.5*(bnvar + 1e-5)**-1.5
dbndiff2 = (torch.ones_like(bndiff2)*1/(n-1)) * dbnvar
dbndiff += 2*bndiff*dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = -dbndiff.sum(0,keepdim=True)
dhprebn += (1/n)*torch.ones_like(hprebn) * dbnmeani
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0, keepdim=True)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [191]:
hprebn.shape, bnmeani.shape

(torch.Size([32, 64]), torch.Size([1, 64]))

In [192]:
dlogits = F.softmax(logits,1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 8.847564458847046e-09


In [193]:
dhprebn = bngain*bnvar_inv * 1/n * ((n*dhpreact) - dhpreact.sum(0) - (n/(n-1)*bnraw*(dhpreact*bnraw).sum(0)))
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [197]:
# let's do the training from the beginning but using my manual & updated gradients

n_embd = 10 # dimensionality of character embedding vectors
n_hidden = 200 # number of neurons in the hidden layer

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(f"Total number of parameters: {sum(p.nelement() for p in parameters)}") # number of parameters in total
for p in parameters:
    p.requires_grad = True

# same optimization as last time
max_steps = 100000
batch_size = 32
n = batch_size # convenience
lossi = []

with torch.no_grad():
    for i in range(max_steps):
        ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
        Xb, Yb = X_train[ix], Y_train[ix]
        
        # forward pass
        emb = C[Xb] # embed the characters into vectors
        embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
        # Linear layer
        hprebn = embcat @ W1 + b1 # hidden layer pre-activation
        # BatchNorm layer
        # -------------------------------------------------------------
        bnmean = hprebn.mean(0, keepdim=True)
        bnvar = hprebn.var(0, keepdim=True, unbiased=True)
        bnvar_inv = (bnvar + 1e-5)**-0.5
        bnraw = (hprebn - bnmean) * bnvar_inv
        hpreact = bngain * bnraw + bnbias
        # -------------------------------------------------------------
        # Non-linearity
        h = torch.tanh(hpreact) # hidden layer
        logits = h @ W2 + b2 # output layer
        loss = F.cross_entropy(logits, Yb) # loss function
    
        
        # backward pass
        for p in parameters:
          p.grad = None
        #loss.backward() # use this for correctness comparisons, delete it later!
    
        # manual backprop! #swole_doge_meme
        dlogits = F.softmax(logits,1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n
        #---2nd layer
        dh = dlogits @ W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0)
        #---tanh
        dhpreact = (1-h**2) * dh
        #---batch norm
        dbngain = (bnraw * dhpreact).sum(0,keepdim=True)
        dbnbias = dhpreact.sum(0,keepdim=True)
        dhprebn = bngain*bnvar_inv * 1/n * ((n*dhpreact) - dhpreact.sum(0) - (n/(n-1)*bnraw*(dhpreact*bnraw).sum(0)))
        #---1st layer
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)
        #---embeddings
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k,j]
                dC[ix] += demb[k,j]
    
        grads =  [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
        #update:
        alpha = 0.1 if i < 100000 else 1e-3
        for param,grad in zip(parameters,grads):
            param.data += -alpha*grad
    
        if i % 10000 == 0: # print every once in a while
            print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())
print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        #if i >= 100:
         #   break

Total number of parameters: 12297
      0/ 100000: 3.7966
  10000/ 100000: 2.1944
  20000/ 100000: 2.3549
  30000/ 100000: 2.3852
  40000/ 100000: 1.9919
  50000/ 100000: 2.4260
  60000/ 100000: 2.3965
  70000/ 100000: 2.0805
  80000/ 100000: 2.3653
  90000/ 100000: 2.1838
  99999/ 100000: 2.0384


In [174]:
# they're all approximately equal to pytorch's gradients. *must have do loss.backward with torch.yes_grad in last cell
#for p,g in zip(parameters, grads):
#    cmp(str(tuple(p.shape)), g, p)

In [198]:
# sample from the model. Same as previous samples.
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carpaizabelle.
khi.
mri.
ree.
caspaessie.
rahne.
fem.
sahti.
reei.
neem.
raice.
iiv.
kalee.
philm.
join.
quinn.
sulio.
alvin.
quinte.
oudiaryxia.
